1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고,
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1500, # 하나의 chunk 가 가질수 있는 토큰 수 
    chunk_overlap=200, # 겹치는 부분 -> 유사도 검색을 할 때 우리가 원하는 문서를 가지고 오는 확률을 높이기 위함함
)

loader = Docx2txtLoader("../tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

ModuleNotFoundError: No module named 'docx2txt'

In [ ]:
%pip install -qU langchain-upstage

In [ ]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

In [7]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# index = pc.Index(index_name)

database = PineconeVectorStore.from_documents(documents=document_list, embedding=embedding, index_name=index_name)


In [8]:
query = '연봉 5000만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query=query, k=3)

In [ ]:
retrived_docs

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [11]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrived_docs}

Question: {query}
"""

In [12]:
ai_message = llm.invoke(prompt)

'연봉 5,000만원인 직장인의 소득세를 계산하기 위해서는 한국의 소득세 과세 표준과 세율을 고려해야 합니다. 2021년 기준으로 한국의 소득세율은 다음과 같습니다:\n\n1. 1,200만원 이하: 6%\n2. 1,200만원 초과 ~ 4,600만원 이하: 15%\n3. 4,600만원 초과 ~ 8,800만원 이하: 24%\n4. 8,800만원 초과 ~ 1억5,000만원 이하: 35%\n5. 1억5,000만원 초과 ~ 3억원 이하: 38%\n6. 3억원 초과 ~ 5억원 이하: 40%\n7. 5억원 초과: 42%\n\n단, 연말정산 시 세액공제, 인적공제, 특별공제 등 여러 공제 항목이 적용되므로 실제 납부해야 하는 세금은 다를 수 있습니다.\n\n연봉 5,000만원의 소득세를 계산할 때 기본적인 세율에 따른 계산 방법은 다음과 같습니다.\n\n1. 1,200만원까지: 1,200만원 * 6% = 72만원\n2. 1,200만원 초과 ~ 4,600만원: (4,600만원 - 1,200만원) * 15% = 510만원\n3. 4,600만원 초과 ~ 5,000만원: (5,000만원 - 4,600만원) * 24% = 96만원\n\n따라서, 총 소득세는 72만원 + 510만원 + 96만원 = 678만원입니다.\n\n위 계산은 기본적인 세율을 기준으로 한 것일 뿐, 실제 세액은 개인의 상황에 따라 달라질 수 있습니다. 정확한 세액은 연말정산 또는 세무 전문가의 상담을 통해 확인하실 수 있습니다.'


In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [14]:

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [15]:
query = '연봉 5000만원인 직장인의 소득세는 얼마인가요?'

In [ ]:
ai_message = qa_chain({"query":query})

In [ ]:
ai_message